In [1]:
!pip install flask

In [2]:
!pip install flask pandas

In [ ]:
###upload dataset and return dataset id###

from flask import Flask, request, jsonify
import os
import pandas as pd
import datetime

app = Flask(__name__)

datasets = {}  # Dictionary to store datasets with unique IDs
log_file = "dataset_log.xlsx"
log_columns = ["dataset_id", "dataset_name", "description", "file_path"]

if not os.path.exists(log_file):
    df = pd.DataFrame(columns=log_columns)
    df.to_excel(log_file, index=False)

def generate_dataset_id():
    return f"ID_{len(datasets) + 1:03d}"  # Generates ID_001, ID_002, etc.

@app.route("/create_dataset", methods=["POST"])
def create_dataset():
    try:
        if 'textfile' not in request.files:
            return jsonify({"error": "Missing textfile parameter"}), 400

        textfile = request.files['textfile']
        content = textfile.read().decode("utf-8").strip().split("\n")
        
        data_dict = {}
        for line in content:
            key, value = line.split(":", 1)
            data_dict[key.strip()] = value.strip()
        
        dataset_name = data_dict.get("dataset_name")
        description = data_dict.get("description")
        file_path = data_dict.get("textfilepath")
        project_type = data_dict.get("project_type")
        
        if not dataset_name or not description or not file_path:
            return jsonify({"error": "Missing required parameters in text file"}), 400
        
        dataset_id = generate_dataset_id()
        datasets[dataset_id] = {
            "datasetName": dataset_name,
            "description": description,
            "filePath": file_path,
            "project_type":project_type
        }

        # Save file in GENAI_HOME folder
        os.makedirs("GENAI_HOME", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        file_name = f"{timestamp}_{os.path.basename(textfile.filename)}_{dataset_id}.txt"
        file_save_path = os.path.join("GENAI_HOME", file_name)
        with open(file_save_path, "w") as f:
            f.write("\n".join(content))

        # Log dataset info to Excel
        df = pd.read_excel(log_file)
        new_row = pd.DataFrame([{"dataset_id": dataset_id, "dataset_name": dataset_name, "description": description, "file_path": file_save_path,"project_type":project_type}])
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_excel(log_file, index=False)

        return jsonify({"datasetId": dataset_id, "message": "Dataset created successfully"}), 201
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Feb/2025 19:57:52] "POST /create_dataset HTTP/1.1" 201 -


In [ ]:
from flask import Flask, request, jsonify
import os
import pandas as pd
import datetime
import random  # ✅ Import the missing module

app = Flask(__name__)

GENAI_HOME = "GENAI_HOME"
os.makedirs(GENAI_HOME, exist_ok=True)

EXCEL_FILE = "metrics_results.xlsx"

def read_config_file(file_path):
    """Reads the text file and extracts parameters."""
    config = {}
    with open(file_path, "r") as f:
        for line in f:
            key, value = line.strip().split(": ", 1)  # Split on ": "
            config[key] = value.strip()
    return config

@app.route("/evaluate_metrics", methods=["POST"])
def evaluate_metrics():
    try:
        # Check if file is uploaded
        if 'configfile' not in request.files:
            return jsonify({"error": "Missing 'configfile' parameter"}), 400

        file = request.files['configfile']

        # Save temporarily & Read file
        temp_file_path = os.path.join(GENAI_HOME, file.filename)
        file.save(temp_file_path)
        config = read_config_file(temp_file_path)

        dataset_id = config.get("dataset_id")
        project_type = config.get("project_type")
        metrics = config.get("metrics", "").split(", ")

        if not dataset_id or not project_type or not metrics:
            return jsonify({"error": "Missing required parameters in config file"}), 400

        # Find the corresponding dataset file
        dataset_files = [f for f in os.listdir(GENAI_HOME) if f.endswith(f"_{dataset_id}.txt")]
        if not dataset_files:
            return jsonify({"error": f"No dataset found for dataset_id: {dataset_id}"}), 404

        dataset_file = os.path.join(GENAI_HOME, dataset_files[0])

        # Simulate metrics evaluation
        results = []
        for metric in metrics:
            score = round(random.uniform(0, 1), 2)  # ✅ Fix: Import random module
            reasoning = f"Placeholder reasoning for {metric}"
            results.append({"dataset_id": dataset_id, "metric": metric, "score": score, "reasoning": reasoning})

        # Store results in Excel
        df = pd.DataFrame(results)
        if os.path.exists(EXCEL_FILE):
            existing_df = pd.read_excel(EXCEL_FILE)
            df = pd.concat([existing_df, df], ignore_index=True)

        df.to_excel(EXCEL_FILE, index=False)

        return jsonify({"status": "success", "message": "Metrics evaluated successfully", "results": results})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2025 17:48:52] "POST /evaluate_metrics HTTP/1.1" 200 -


In [ ]:
import os
import logging
import json
import pandas as pd
from flask import Flask, request, jsonify
from openai import AzureOpenAI
from datetime import datetime

app = Flask(__name__)

# Configuration
GEN_AI_HOME = "C://Users//UTHRAVFST//GENAI_HOME"  # Directory where datasets are stored

api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize OpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

logging.basicConfig(level=logging.INFO)


def find_dataset(dataset_id):
    """Finds the dataset file based on dataset_id in GEN_AI_HOME."""
    for file in os.listdir(GEN_AI_HOME):
        if file.endswith(f"{dataset_id}.txt"):
            return os.path.join(GEN_AI_HOME, file)
    return None


def brevity_augmentation(text):
    """Uses OpenAI API to summarize the text (Brevity Augment)."""
    brevity_prompt = f"""You are an AI expert in text summarization. Your task is to make the given text concise while preserving its key meaning. 
    Please shorten the following text:
    {text}"""

    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[{"role": "system", "content": "You are a summarization assistant."},
                      {"role": "user", "content": brevity_prompt}],
            max_tokens=1000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Error while executing brevity augmentation: {e}")
        return None


def store_results(dataset_id, augment, transformed_text):
    """Stores augmentation results in an Excel file."""
    result_file = os.path.join(GEN_AI_HOME, "augmentation_results.xlsx")

    data = {
        "Dataset ID": [dataset_id],
        "Augment": [augment],
        "Transformed Text": [transformed_text],
        "Timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    }

    df = pd.DataFrame(data)

    if os.path.exists(result_file):
        existing_df = pd.read_excel(result_file)
        df = pd.concat([existing_df, df], ignore_index=True)

    df.to_excel(result_file, index=False)
@app.route("/execute_augment", methods=["POST"])
def execute_augment():
    """API to execute augmentation on a dataset."""
    try:
        config = request.files.get("config")
        if not config:
            return jsonify({"error": "Missing config file"}), 400

        # Read the config file as text
        try:
            config_text = config.read().decode("utf-8").strip()
            config_data = {}

            for line in config_text.split("\n"):
                line = line.strip()
                if not line or ":" not in line:  # Ignore empty lines or invalid format
                    continue
                
                key, value = line.split(":", 1)
                config_data[key.strip()] = value.strip()

        except Exception as e:
            logging.error(f"Error parsing config file: {e}")
            return jsonify({"error": "Invalid config file format"}), 400

        dataset_id = config_data.get("dataset_id")
        project_type = config_data.get("project_type")
        augments_list = config_data.get("augments_list")

        if not dataset_id or not project_type or not augments_list:
            return jsonify({"error": "Config file missing required fields"}), 400

        # Ensure augments_list is a list
        augments_list = [augment.strip() for augment in augments_list.split(",")]

        dataset_path = find_dataset(dataset_id)
        if not dataset_path:
            return jsonify({"error": f"Dataset with ID {dataset_id} not found"}), 404

        with open(dataset_path, "r", encoding="utf-8") as file:
            dataset_text = file.read()

        results = {}
        for augment in augments_list:
            if augment.lower() == "brevity":
                transformed_text = brevity_augmentation(dataset_text)
                if transformed_text:
                    store_results(dataset_id, augment, transformed_text)
                    results[augment] = transformed_text
                else:
                    results[augment] = "Augmentation failed"

        return jsonify({"dataset_id": dataset_id, "results": results})

    except Exception as e:
        logging.error(f"Error processing augmentation: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

 
if __name__ == "__main__":
    app.run(debug=False, use_reloader=False, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:httpx:HTTP Request: POST https://tcoeaiteamgpt4o.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:werkzeug:127.0.0.1 - - [19/Feb/2025 18:21:48] "POST /execute_augment HTTP/1.1" 200 -


In [ ]:
import os
import logging
import json
import pandas as pd
from flask import Flask, request, jsonify
from openai import AzureOpenAI
from datetime import datetime

app = Flask(__name__)

# Configuration
GEN_AI_HOME = "C://Users//UTHRAVFST//GEN_AI_HOME"  # Root directory for storing datasets

api_base = os.getenv("API_BASE", "https://tcoeaiteamgpt4o.openai.azure.com/")
api_key = os.getenv("API_KEY", "DH8RAnW9UnbzuSxm8luI7Go8nX42bdLhFIEKcPQ1U0JpOU0hPeLKJQQJ99BAACYeBjFXJ3w3AAABACOGdstk")
deployment_name = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = os.getenv("API_VERSION", "2024-05-01-preview")

# Initialize OpenAI client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}",
)

logging.basicConfig(level=logging.INFO)

def parse_config(config_file):
    """Parses the text-based config file into a dictionary."""
    config_data = {}
    try:
        lines = config_file.read().decode("utf-8").splitlines()  # Decode bytes to string
        for line in lines:
            if "=" in line:  # Ensure correct key-value format
                key, value = line.strip().split("=", 1)
                config_data[key.strip()] = value.strip()
        return config_data
    except Exception as e:
        logging.error(f"Error parsing config file: {e}")
        return None

def create_dataset(config_data):
    """Creates the dataset inside the GEN_AI_HOME directory."""
    project_type = config_data["project_type"]
    textfilepath = config_data["textfilepath"]
    dataset_name = config_data["dataset_name"]
    dataset_id = datetime.now().strftime("%Y%m%d%H%M%S")
    
    project_folder = os.path.join(GEN_AI_HOME, project_type)
    os.makedirs(project_folder, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    new_filename = f"{timestamp}_{os.path.basename(textfilepath)}_{dataset_id}.txt"
    dataset_path = os.path.join(project_folder, new_filename)
    
    with open(textfilepath, "r", encoding="utf-8") as src_file, open(dataset_path, "w", encoding="utf-8") as dest_file:
        dest_file.write(src_file.read())
    
    return dataset_id, dataset_path

def brevity_augmentation(text):
    """Uses OpenAI API to summarize the text (Brevity Augment)."""
    brevity_prompt = f"Shorten the following text while preserving key meaning: {text}"
    return generate_openai_response(brevity_prompt)

def verbosity_augmentation(text):
    """Uses OpenAI API to expand the text with more details (Verbosity Augment)."""
    verbosity_prompt = f"Expand and elaborate the following text with more details: {text}"
    return generate_openai_response(verbosity_prompt)

def generate_openai_response(prompt):
    """Helper function to interact with OpenAI API."""
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[{"role": "system", "content": "You are an AI assistant."},
                      {"role": "user", "content": prompt}],
            max_tokens=1000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f"Error while processing augmentation: {e}")
        return None

def store_results_excel(dataset_id, dataset_name, project_type, augment, transformed_text, metric, score, reasoning):
    """Stores results in an Excel file."""
    result_file = os.path.join(GEN_AI_HOME, f"{dataset_id}_{dataset_name}.xlsx")
    
    data = {
        "Dataset ID": [dataset_id],
        "Dataset Name": [dataset_name],
        "Project Type": [project_type],
        "Augmentation": [augment],
        "Transformed Text": [transformed_text],
        "Metric": [metric],
        "Score": [score],
        "Reasoning": [reasoning],
        "Timestamp": [datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
    }
    df = pd.DataFrame(data)
    
    if os.path.exists(result_file):
        existing_df = pd.read_excel(result_file)
        df = pd.concat([existing_df, df], ignore_index=True)
    
    df.to_excel(result_file, index=False)

@app.route("/process", methods=["POST"])
def process_pipeline():
    """Main API endpoint to process dataset, apply augments, and evaluate metrics."""
    try:
        config_file = request.files.get("config")
        if not config_file:
            return jsonify({"error": "Missing config file"}), 400
        
        config_data = parse_config(config_file)
        if not config_data:
            return jsonify({"error": "Invalid config file format"}), 400
        
        dataset_id, dataset_path = create_dataset(config_data)
        
        augments_list = config_data.get("augments", "").split(",")
        metrics_list = config_data.get("metrics", "").split(",")
        dataset_text = open(dataset_path, "r", encoding="utf-8").read()
        
        augmentations_folder = os.path.join(os.path.dirname(dataset_path), f"augmentations_{dataset_id}")
        os.makedirs(augmentations_folder, exist_ok=True)
        
        augmentation_results = {}
        for augment in augments_list:
            transformed_text = None
            if augment.lower() == "brevity":
                transformed_text = brevity_augmentation(dataset_text)
            elif augment.lower() == "verbosity":
                transformed_text = verbosity_augmentation(dataset_text)
            
            if transformed_text:
                augment_file = os.path.join(augmentations_folder, f"{augment}.txt")
                with open(augment_file, "w", encoding="utf-8") as f:
                    f.write(transformed_text)
                augmentation_results[augment] = transformed_text
        
        for augment, text in augmentation_results.items():
            for metric in metrics_list:
                score = "NA"  # Replace with actual metric scoring function
                reasoning = "Metric evaluation logic goes here."
                store_results_excel(dataset_id, config_data["dataset_name"], config_data["project_type"], augment, text, metric, score, reasoning)
        
        return jsonify({"dataset_id": dataset_id, "results": augmentation_results})
    except Exception as e:
        logging.error(f"Processing error: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Feb/2025 20:41:54] "POST /process HTTP/1.1" 200 -
